# Create and Train our Transformer

### Load the Dataset

In [1]:
from datasets import load_dataset


# Load the dataset
train_dataset, test_dataset = load_dataset("dataset/PokedexDataset.py", split=["train", "test"])

Reusing dataset pokedex_dataset (C:\Users\Will Sumerfield\.cache\huggingface\datasets\pokedex_dataset\plain_text\1.0.0\3aa69cafe54b66fd5280fcf12671ebf466fe770853e5254944dd13cefd7a6698)


### Tokenize the Dataset

In [2]:
from transformers import GPT2Tokenizer


# Create the tokenizer instance
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the padding token to use
tokenizer.pad_token = tokenizer.eos_token


def tokenize_data(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)


# Tokenize our datasets
tokenized_train_dataset = train_dataset.map(tokenize_data, batched=True)
tokenized_test_dataset =  test_dataset.map(tokenize_data, batched=True)

Loading cached processed dataset at C:\Users\Will Sumerfield\.cache\huggingface\datasets\pokedex_dataset\plain_text\1.0.0\3aa69cafe54b66fd5280fcf12671ebf466fe770853e5254944dd13cefd7a6698\cache-174e655c7e0f1d84.arrow
Loading cached processed dataset at C:\Users\Will Sumerfield\.cache\huggingface\datasets\pokedex_dataset\plain_text\1.0.0\3aa69cafe54b66fd5280fcf12671ebf466fe770853e5254944dd13cefd7a6698\cache-9bb7031b592819bf.arrow


### Import our Model

In [3]:
import torch
from transformers import GPT2Model


# Get a pretrained instance of the model
model = GPT2Model.from_pretrained('gpt2').to('cuda:0')

### Train the Model

In [4]:
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from datasets import load_metric


# Store the metric for accuracy
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    """This method is used to compute to find the accuracy of our model"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Set the parameters for the trainer
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

# Create a trainer with the provided arguments
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_train_dataset,
                  eval_dataset=tokenized_test_dataset,
                  compute_metrics=compute_metrics)

In [5]:
# Train the Model
trainer.train()


The following columns in the training set  don't have a corresponding argument in `GPT2Model.forward` and have been ignored: text.
C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3072
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1152


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 8.00 GiB total capacity; 6.55 GiB already allocated; 0 bytes free; 6.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF